In [1]:
import numpy as np
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from sklearn.model_selection import train_test_split

In [2]:
X = np.load('data/games_ar.npy', allow_pickle=True)
y = np.load('data/winner.npy', allow_pickle=True)

In [3]:
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.1)

In [4]:
train_X = torch.tensor(train_X, dtype=torch.float)
train_y = torch.tensor(train_y, dtype=torch.long)
test_X = torch.tensor(test_X, dtype=torch.float)
test_y = torch.tensor(test_y, dtype=torch.long)

In [5]:
features = [nn.Linear(train_X.shape[1], 256),
    nn.ReLU(),
    nn.Dropout(p=0.2),
    nn.Linear(256, 256),
    nn.ReLU(),
    nn.Dropout(p=0.2),
    nn.Linear(256,2)]

In [6]:
net = nn.Sequential(*features).cuda()

In [7]:
optimizer = optim.Adam(net.parameters(), lr=0.001)
loss_fn = nn.CrossEntropyLoss()

In [8]:
def train(net, X, y, EPOCHS=8, BATCH_SIZE=100):
    for epoch in range(EPOCHS):
        for i in tqdm(range(0, len(X), BATCH_SIZE)):
            batch_X = X[i:i+BATCH_SIZE].cuda()
            batch_y = y[i:i+BATCH_SIZE].cuda()
            
            net.zero_grad()
            output = net(batch_X)
            loss = loss_fn(output, batch_y)
            loss.backward()
            optimizer.step()
        print(loss)

In [9]:
train(net, train_X, train_y)

100%|█████████████████████████████████████████████████████████████████████████████████| 14/14 [00:00<00:00, 478.07it/s]


tensor(0.7160, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.6811, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.6071, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.4677, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.3295, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.2019, device='cuda:0', grad_fn=<NllLossBackward>)

100%|█████████████████████████████████████████████████████████████████████████████████| 14/14 [00:00<00:00, 462.46it/s]


tensor(0.0902, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.0322, device='cuda:0', grad_fn=<NllLossBackward>)


In [10]:
def test(net, X, y):
    correct = 0
    total = 0
    net.eval()
    with torch.no_grad():
        output = net(X.cuda())
        for idx, i in enumerate(output):
            if torch.argmax(i) == y[idx]:
                correct += 1
            total += 1
    net.train()
    return round(correct/total,3)

In [11]:
print(f"Train accuracy: {test(net, train_X, train_y)}")
print(f"Test accuracy: {test(net, test_X, test_y)}")

Train accuracy: 0.992
Test accuracy: 0.591


In [12]:
# overfitted but 60% over 150 games is significantly better than guessing
test_X.shape[0]

149

In [13]:
def save_model():
    torch.save(net.state_dict(), 'models/sixty.pth')

In [14]:
def load_model():
    model = nn.Sequential(*features).cuda()
    model.load_state_dict(torch.load('models/sixty.pth'))
    model.eval()
    return model

In [15]:
save_model()